In [1]:
import torch 
import torchvision
import tqdm
from torchvision import transforms as transforms
import os
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
dataset = torchvision.datasets.CIFAR10
data_dir = '/content/drive/My Drive/task2/data'
batch_size = 125
num_workers = 20
device = torch.device('cuda')
cuda = device.type == 'cuda'
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
])
train_dataset = dataset(data_dir, train=True, download=True, transform=transform)
train_iter = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size,
                                         shuffle=True, num_workers=num_workers, pin_memory=cuda)

val_dataset = dataset(data_dir, download=True, train=False, transform=transform)
val_iter = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers,
                                       pin_memory=cuda)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet18', pretrained=False)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.6.0


In [5]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [6]:
model.fc = torch.nn.Linear(512, 10)

In [7]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [8]:
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [9]:
loss = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-1, momentum=9e-1)

In [10]:


def train_network(network, loss, optimizer, train_iter, val_iter, num_epochs, device='cpu', start_epoch=0,
                 checkpoints=False, out_dir=None):
    """Model training"""
    training_cycles = dict(loss=[], acc=[], val_loss=[], val_acc=[])
    best_epoch, best_acc = 0, 0.0
    
    
    for epoch_ in range(start_epoch, start_epoch+num_epochs):
        
        print("Epoch %d "% (epoch_+1))
        
        
        ## Training 
        print("Training ", flush=True)
        correct, avg_loss, total = 0, 0, 0 
        network.train()
        data_iter = tqdm.tqdm(train_iter, desc="Loss: %.3f, Acc:%.3f"% (0, 0))
        
        
        for batch_x , batch_y in data_iter:
            optimizer.zero_grad() # initialized zero gradient
            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)
            
            # forward propogation
            res = network(batch_x) 
            # loss 
            l = loss(res, batch_y)
            # back progation
            l.backward()
            optimizer.step()
            res = torch.argmax(res, dim=-1)
            correct += torch.sum(res == batch_y).data.cpu().numpy()
            total += batch_x.shape[0]
            avg_loss += l.data.cpu().numpy() * batch_x.shape[0]
            data_iter.set_description('Loss: %.3f, Acc: %.3f' % (avg_loss / total, correct / total))

        training_cycles['loss'].append(avg_loss/total)
        training_cycles['acc'].append(correct / total)
        
        ## Validation 
        print("Validation ", flush=True)
        network.eval()
        correct, avg_loss, total = 0, 0, 0
        data_iter = tqdm.tqdm(val_iter, desc="Loss: %.3f, Acc:%.3f" %(0, 0))
        
        with torch.no_grad():
            for batch_x, batch_y in data_iter:
                batch_x = batch_x.to(device)
                batch_y = batch_y.to(device)
                res = network(batch_x)
                l = loss(res, batch_y)
                res = torch.argmax(res, dim=-1)
                correct += torch.sum(res == batch_y).data.cpu().numpy()
                total += batch_x.shape[0]
                avg_loss += l.data.cpu().numpy() * batch_x.shape[0]
                data_iter.set_description('Loss: %.3f, Acc: %.3f' % (avg_loss / total, correct / total))
        training_cycles['val_loss'].append(avg_loss/total)
        training_cycles['val_acc'].append(correct/total)  
        if (correct/total) > best_acc:
            best_epoch = epoch_
            best_acc = correct/total
        if checkpoints and out_dir is not None:
            if not os.path.isdir(out_dir):
                os.makedirs(out_dirs)
            save_path = os.path.join(out_dir, 'model_.pth') # saving only last epoch 
            torch.save(network.state_dict(), save_path)
    
    print("Best epoch %d , validation _accuracy %.2f" %(best_epoch+1, best_acc*100))
    return training_cycles

In [11]:
load_path = None
checkpoints= True
out_dirs = '/content/drive/My Drive/task2/model'



start_epoch = 0

if load_path is not None:
    state_dict = torch.load(load_path)
    model.load_state_dict(state_dict)
else:
    train_network(network=model, loss=loss, optimizer=optimizer, train_iter=train_iter, val_iter=val_iter,
                  num_epochs=10, device=device, start_epoch=start_epoch, checkpoints=checkpoints, 
                  out_dir=out_dirs)



Epoch 1 
Training 


Loss: 2.067, Acc: 0.281: 100%|██████████| 400/400 [02:35<00:00,  2.57it/s]

Validation 



Loss: 1.730, Acc: 0.366: 100%|██████████| 80/80 [00:20<00:00,  4.00it/s]

Epoch 2 
Training 


...

Epoch 10 
Training 


Loss: 0.194, Acc: 0.931: 100%|██████████| 400/400 [02:32<00:00,  2.62it/s]

Validation 



Loss: 0.721, Acc: 0.791: 100%|██████████| 80/80 [00:19<00:00,  4.00it/s]


Best epoch 9 , validation _accuracy 81.26


In [12]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

**Full Model: Accuracy report **

In [13]:
for batch_x, batch_y in val_iter:
    batch_x = batch_x.to(device)
    labels = batch_y.cpu().data.numpy()
    predicted_labels = model(batch_x)
    predicted_labels = torch.argmax(predicted_labels, dim=-1)


In [14]:
predicted_labels = np.squeeze(predicted_labels.cpu().data.numpy())

In [15]:

print(confusion_matrix(labels, predicted_labels))
print(classification_report(labels, predicted_labels))

[[ 5  0  0  0  0  0  0  0  1  0]
 [ 0  8  0  0  1  0  0  0  0  0]
 [ 0  0 12  2  0  0  0  1  0  0]
 [ 0  0  1 16  0  0  0  0  0  0]
 [ 0  0  2  1  8  0  0  1  0  0]
 [ 0  0  1  8  0  6  0  2  0  0]
 [ 0  0  1  1  2  0 12  0  0  0]
 [ 0  0  1  0  0  1  0  8  0  0]
 [ 0  1  0  0  0  0  0  0  4  0]
 [ 0  0  1  0  0  0  0  0  0 17]]
              precision    recall  f1-score   support

           0       1.00      0.83      0.91         6
           1       0.89      0.89      0.89         9
           2       0.63      0.80      0.71        15
           3       0.57      0.94      0.71        17
           4       0.73      0.67      0.70        12
           5       0.86      0.35      0.50        17
           6       1.00      0.75      0.86        16
           7       0.67      0.80      0.73        10
           8       0.80      0.80      0.80         5
           9       1.00      0.94      0.97        18

    accuracy                           0.77       125
   macro avg       

In [16]:

def recall_(labels, predicted_labels, class_):
  
  total = np.count_nonzero(labels == class_)
  count = 0
  for i in range(len(labels)):
    if labels[i] == predicted_labels[i] == class_:
      count+=1
  return count/total

In [17]:
model_1_report = dict()

for c in range(10):
  model_1_report[c] = recall_(labels, predicted_labels, c)

In [18]:
model_1_report

{0: 0.8333333333333334,
 1: 0.8888888888888888,
 2: 0.8,
 3: 0.9411764705882353,
 4: 0.6666666666666666,
 5: 0.35294117647058826,
 6: 0.75,
 7: 0.8,
 8: 0.8,
 9: 0.9444444444444444}

### Model 2 :: 
Using only the red channel as input (and  making a 3D tensor by copying the data of the red channel in the blue and green channels


In [19]:
dataset = torchvision.datasets.CIFAR10
data_dir = '/content/drive/My Drive/task2/dataR'

def copy_red_channel(data_object):
    data_object[1], data_object[2] = data_object[0], data_object[0]
    return data_object


transform_red = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Lambda(copy_red_channel)
])

train_R_dataset = dataset(data_dir, train=True, download=True, transform=transform_red)
train_R_iter = torch.utils.data.DataLoader(train_R_dataset, batch_size=batch_size,
                                         shuffle=True, num_workers=num_workers, pin_memory=cuda)

val_R_dataset = dataset(data_dir, download=True, train=False, transform=transform)
val_R_iter = torch.utils.data.DataLoader(val_R_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers,
                                       pin_memory=cuda)

Files already downloaded and verified
Files already downloaded and verified


In [20]:
model_R = torch.hub.load('pytorch/vision:v0.6.0', 'resnet18', pretrained=False)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.6.0


In [21]:
model_R.fc = torch.nn.Linear(512, 10)

In [22]:
model_R.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [23]:
loss = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model_R.parameters(), lr=1e-1, momentum=9e-1)

In [24]:
load_path = None
checkpoints= True
out_dirs = '/content/drive/My Drive/task2/model_R'



start_epoch = 0

if load_path is not None:
    state_dict = torch.load(load_path)
    model.load_state_dict(state_dict)
else:
    train_network(network=model_R, loss=loss, optimizer=optimizer, train_iter=train_R_iter, val_iter=val_R_iter,
                  num_epochs=10, device=device, start_epoch=start_epoch, checkpoints=checkpoints, 
                  out_dir=out_dirs)

Epoch 1 
Training 


Loss: 2.250, Acc: 0.193: 100%|██████████| 400/400 [02:35<00:00,  2.58it/s]

Validation 



Loss: 2.154, Acc: 0.241: 100%|██████████| 80/80 [00:20<00:00,  3.96it/s]


Epoch 2 
Training 


...

Epoch 10 
Training 



Loss: 0.112, Acc: 0.962: 100%|██████████| 400/400 [02:32<00:00,  2.61it/s]

Validation 



Loss: 0.927, Acc: 0.764: 100%|██████████| 80/80 [00:19<00:00,  4.09it/s]


Best epoch 9 , validation _accuracy 78.23


In [25]:
for batch_x, batch_y in val_R_iter:
    batch_x = batch_x.to(device)
    labels = batch_y.cpu().data.numpy()
    predicted_labels = model_R(batch_x)
    predicted_labels = torch.argmax(predicted_labels, dim=-1)

In [26]:
model_R_report = dict()

for c in range(10):
  model_R_report[c] = recall_(labels, predicted_labels, c)

In [27]:
model_R_report

{0: 1.0,
 1: 0.8888888888888888,
 2: 0.6,
 3: 0.9333333333333333,
 4: 0.8333333333333334,
 5: 0.3,
 6: 0.7692307692307693,
 7: 0.9411764705882353,
 8: 0.75,
 9: 0.9166666666666666}

Model 3 : Using only Green Channel

In [28]:


dataset = torchvision.datasets.CIFAR10
data_dir = '/content/drive/My Drive/task2/dataG'

def copy_green_channel(data_object):
    data_object[0], data_object[2] = data_object[1], data_object[1]
    return data_object


transform_green = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Lambda(copy_green_channel)
])

train_G_dataset = dataset(data_dir, train=True, download=True, transform=transform_green)
train_G_iter = torch.utils.data.DataLoader(train_G_dataset, batch_size=batch_size,
                                         shuffle=True, num_workers=num_workers, pin_memory=cuda)

val_G_dataset = dataset(data_dir, download=True, train=False, transform=transform)
val_G_iter = torch.utils.data.DataLoader(val_G_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers,
                                       pin_memory=cuda)


Files already downloaded and verified
Files already downloaded and verified


In [29]:
model_G = torch.hub.load('pytorch/vision:v0.6.0', 'resnet18', pretrained=False)
model_G.fc = torch.nn.Linear(512, 10)
model_G.to(device)
loss = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model_G.parameters(), lr=1e-1, momentum=9e-1)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.6.0


In [30]:
load_path = None
checkpoints= True
out_dirs = '/content/drive/My Drive/task2/model_G'



start_epoch = 0

if load_path is not None:
    state_dict = torch.load(load_path)
    model.load_state_dict(state_dict)
else:
    train_network(network=model_G, loss=loss, optimizer=optimizer, train_iter=train_G_iter, val_iter=val_G_iter,
                  num_epochs=10, device=device, start_epoch=start_epoch, checkpoints=checkpoints, 
                  out_dir=out_dirs)

Epoch 1 
Training 


Loss: 2.172, Acc: 0.248: 100%|██████████| 400/400 [02:33<00:00,  2.60it/s]

Validation 



Loss: 2.309, Acc: 0.237: 100%|██████████| 80/80 [00:19<00:00,  4.07it/s]

Epoch 2 
Training 


...

Epoch 10 
Training 



Loss: 0.162, Acc: 0.945: 100%|██████████| 400/400 [02:32<00:00,  2.61it/s]

Validation 



Loss: 1.097, Acc: 0.715: 100%|██████████| 80/80 [00:19<00:00,  4.07it/s]

Best epoch 7 , validation _accuracy 75.62


In [31]:
for batch_x, batch_y in val_G_iter:
    batch_x = batch_x.to(device)
    labels = batch_y.cpu().data.numpy()
    predicted_labels = model_G(batch_x)
    predicted_labels = torch.argmax(predicted_labels, dim=-1)

In [32]:
model_G_report = dict()

for c in range(10):
  model_G_report[c] = recall_(labels, predicted_labels, c)

In [33]:
model_G_report

{0: 0.7272727272727273,
 1: 0.6363636363636364,
 2: 0.7272727272727273,
 3: 0.5384615384615384,
 4: 0.5,
 5: 1.0,
 6: 0.5625,
 7: 0.8181818181818182,
 8: 0.6875,
 9: 1.0}

Model 4: Using only Blue Channel

In [34]:
dataset = torchvision.datasets.CIFAR10
data_dir = '/content/drive/My Drive/task2/dataB'

def copy_blue_channel(data_object):
    data_object[0], data_object[1] = data_object[2], data_object[2]
    return data_object


transform_blue = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Lambda(copy_blue_channel)
])

train_B_dataset = dataset(data_dir, train=True, download=True, transform=transform_blue)
train_B_iter = torch.utils.data.DataLoader(train_B_dataset, batch_size=batch_size,
                                           shuffle=True, num_workers=num_workers, pin_memory=cuda)

val_B_dataset = dataset(data_dir, download=True, train=False, transform=transform)
val_B_iter = torch.utils.data.DataLoader(val_B_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers,
                                         pin_memory=cuda)

Files already downloaded and verified
Files already downloaded and verified


In [35]:
model_B = torch.hub.load('pytorch/vision:v0.6.0', 'resnet18', pretrained=False)
model_B.fc = torch.nn.Linear(512, 10)
model_B.to(device)
loss = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model_B.parameters(), lr=1e-1, momentum=9e-1)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.6.0


In [36]:
load_path = None
checkpoints= True
out_dirs = '/content/drive/My Drive/task2/model_B'

start_epoch = 0

if load_path is not None:
    state_dict = torch.load(load_path)
    model.load_state_dict(state_dict)
else:
    train_network(network=model_B, loss=loss, optimizer=optimizer, train_iter=train_B_iter, val_iter=val_B_iter,
                  num_epochs=10, device=device, start_epoch=start_epoch, checkpoints=checkpoints, 
                  out_dir=out_dirs)

Epoch 1 
Training 


Loss: 1.997, Acc: 0.299: 100%|██████████| 400/400 [02:34<00:00,  2.58it/s]

Validation 



Loss: 1.710, Acc: 0.371: 100%|██████████| 80/80 [00:19<00:00,  4.06it/s]


Epoch 2 
Training 


...

Epoch 10 
Training 


Loss: 0.090, Acc: 0.970: 100%|██████████| 400/400 [02:35<00:00,  2.58it/s]

Validation 



Loss: 0.969, Acc: 0.774: 100%|██████████| 80/80 [00:19<00:00,  4.01it/s]


Best epoch 10 , validation _accuracy 77.42


In [37]:
for batch_x, batch_y in val_B_iter:
    batch_x = batch_x.to(device)
    labels = batch_y.cpu().data.numpy()
    predicted_labels = model_B(batch_x)
    predicted_labels = torch.argmax(predicted_labels, dim=-1)

In [38]:
model_B_report = dict()

for c in range(10):
  model_B_report[c] = recall_(labels, predicted_labels, c)

In [39]:
model_B_report

{0: 0.75,
 1: 0.8333333333333334,
 2: 0.6153846153846154,
 3: 1.0,
 4: 0.7692307692307693,
 5: 0.3,
 6: 0.8571428571428571,
 7: 1.0,
 8: 0.9,
 9: 0.9}

In [40]:
import pandas as pd

In [41]:

columns = ['Model ', 'Model R', 'Model G','Model B' ]
cell_text = []
cell_text.append(list(model_1_report.values()))
cell_text.append(list(model_R_report.values()))
cell_text.append(list(model_G_report.values()))
cell_text.append(list(model_B_report.values()))
cell_text = np.transpose(cell_text)
pd.DataFrame(cell_text, columns=columns)


,Model,Model R,Model G,Model B
0,0.833333,1.000000,0.727273,0.750000
1,0.888889,0.888889,0.636364,0.833333
2,0.800000,0.600000,0.727273,0.615385
3,0.941176,0.933333,0.538462,1.000000
4,0.666667,0.833333,0.500000,0.769231
5,0.352941,0.300000,1.000000,0.300000
6,0.750000,0.769231,0.562500,0.857143
7,0.800000,0.941176,0.818182,1.000000
8,0.800000,0.750000,0.687500,0.900000
9,0.944444,0.916667,1.000000,0.900000
